In [ ]:
!pip install rdkit-pypi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 47.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams

from rdkit.Avalon import pyAvalonTools
from rdkit import Chem
from rdkit.Chem import Crippen, Descriptors, GraphDescriptors, Lipinski, QED, rdMolDescriptors
from tqdm import tqdm

In [ ]:
import sklearn
from sklearn.feature_selection import SelectKBest, mutual_info_classif


In [ ]:
df = pd.read_csv('classification_dataset_without_descriptors.csv')
df.head()

,SMILES,Activity,Agglomeration,SMILES_uncharge,FORMAL_CHARGE_unch
0,BrC(Br)Br,1,False,BrC(Br)Br,0
1,C#CC(C)(O)CC,1,False,C#CC(C)(O)CC,0
2,C#CC(O)(/C=C/Cl)CC,1,False,C#CC(O)(/C=C/Cl)CC,0
3,C#CC(OC(N)=O)c1ccccc1,1,False,C#CC(OC(N)=O)C1=CC=CC=C1,0
4,C#CC1(OCC(O)CN2CCN(c3ccc(F)cc3)CC2)CCCCC1,1,False,C#CC1(OCC(O)CN2CCN(C3=CC=C(F)C=C3)CC2)CCCCC1,0


In [ ]:
df = df[['SMILES_uncharge', 'Activity']]


In [ ]:
df = df.rename(columns={'SMILES_uncharge':'SMILES'})
df.head()

,SMILES,Activity
0,BrC(Br)Br,1
1,C#CC(C)(O)CC,1
2,C#CC(O)(/C=C/Cl)CC,1
3,C#CC(OC(N)=O)C1=CC=CC=C1,1
4,C#CC1(OCC(O)CN2CCN(C3=CC=C(F)C=C3)CC2)CCCCC1,1


### QED and Crippen modules

In [ ]:
for i, row in df.iterrows():
    mol = Chem.MolFromSmiles(row.SMILES)
    qed_vector = QED.properties(mol)
    df.at[i, 'MW'] = round(qed_vector[0], 2)
    df.at[i, '#HBA'] = qed_vector[2]
    df.at[i, '#HBD'] = qed_vector[3]
    df.at[i, 'PSA'] = qed_vector[4]
    df.at[i, '#ROTB'] = qed_vector[5]
    df.at[i, '#ALERTS'] = qed_vector[7]

    df.at[i, 'MlogP'] = round(Crippen.MolLogP(mol), 2)
    df.at[i, '#MR'] = round(Crippen.MolMR(mol), 2)

df.head()

,SMILES,Activity,MW,#HBA,#HBD,PSA,#ROTB,#ALERTS,MlogP,#MR
0,BrC(Br)Br,1,252.73,0.0,0.0,0.00,0.0,1.0,2.45,30.57
1,C#CC(C)(O)CC,1,98.14,1.0,1.0,20.23,1.0,1.0,0.78,29.75
2,C#CC(O)(/C=C/Cl)CC,1,144.60,1.0,1.0,20.23,2.0,1.0,1.51,39.07
3,C#CC(OC(N)=O)C1=CC=CC=C1,1,175.19,2.0,1.0,52.32,2.0,1.0,1.46,48.68
4,C#CC1(OCC(O)CN2CCN(C3=CC=C(F)C=C3)CC2)CCCCC1,1,360.47,4.0,1.0,35.94,6.0,1.0,2.66,101.66


### Lipinski module

In [ ]:
for i, row in df.iterrows():
    mol = Chem.MolFromSmiles(row.SMILES)
    df.at[i, '#HeavyAtoms'] = Lipinski.HeavyAtomCount(mol)
    df.at[i, '#NHOH'] = Lipinski.NHOHCount(mol)
    df.at[i, '#NO'] = Lipinski.NOCount(mol)
    df.at[i, '#AromaticCarbocycles'] = Lipinski.NumAromaticCarbocycles(mol)
    df.at[i, '#AromaticHeterocycles'] = Lipinski.NumAromaticHeterocycles(mol)
    df.at[i, '#Heteroatoms'] = Lipinski.NumHeteroatoms(mol)

df.head()

,SMILES,Activity,MW,#HBA,#HBD,PSA,#ROTB,#ALERTS,MlogP,#MR,#HeavyAtoms,#NHOH,#NO,#AromaticCarbocycles,#AromaticHeterocycles,#Heteroatoms
0,BrC(Br)Br,1,252.73,0.0,0.0,0.00,0.0,1.0,2.45,30.57,4.0,0.0,0.0,0.0,0.0,3.0
1,C#CC(C)(O)CC,1,98.14,1.0,1.0,20.23,1.0,1.0,0.78,29.75,7.0,1.0,1.0,0.0,0.0,1.0
2,C#CC(O)(/C=C/Cl)CC,1,144.60,1.0,1.0,20.23,2.0,1.0,1.51,39.07,9.0,1.0,1.0,0.0,0.0,2.0
3,C#CC(OC(N)=O)C1=CC=CC=C1,1,175.19,2.0,1.0,52.32,2.0,1.0,1.46,48.68,13.0,2.0,3.0,1.0,0.0,3.0
4,C#CC1(OCC(O)CN2CCN(C3=CC=C(F)C=C3)CC2)CCCCC1,1,360.47,4.0,1.0,35.94,6.0,1.0,2.66,101.66,26.0,1.0,4.0,1.0,0.0,5.0


### Descriptors module

In [ ]:
for i, row in df.iterrows():
    mol = Chem.MolFromSmiles(row.SMILES)
    df.at[i, 'Morgan2'] =  round(Descriptors.FpDensityMorgan2(mol), 2)
    df.at[i, 'Morgan3'] =  round(Descriptors.FpDensityMorgan3(mol), 2)
    df.at[i, 'HeavyAtomMW'] =  round(Descriptors.HeavyAtomMolWt(mol), 2)
    df.at[i, 'MaxPartialCharge'] = Descriptors.MaxPartialCharge(mol)
    df.at[i, 'MinPartialCharge'] = Descriptors.MinPartialCharge(mol)
    df.at[i, '#ValenceElectrons'] = Descriptors.NumValenceElectrons(mol)

df.head()

,SMILES,Activity,MW,#HBA,#HBD,PSA,#ROTB,#ALERTS,MlogP,#MR,...,#NO,#AromaticCarbocycles,#AromaticHeterocycles,#Heteroatoms,Morgan2,Morgan3,HeavyAtomMW,MaxPartialCharge,MinPartialCharge,#ValenceElectrons
0,BrC(Br)Br,1,252.73,0.0,0.0,0.00,0.0,1.0,2.45,30.57,...,0.0,0.0,0.0,3.0,1.00,1.00,251.72,0.124221,-0.063717,26.0
1,C#CC(C)(O)CC,1,98.14,1.0,1.0,20.23,1.0,1.0,0.78,29.75,...,1.0,0.0,0.0,1.0,2.29,2.29,88.06,0.121725,-0.377933,40.0
2,C#CC(O)(/C=C/Cl)CC,1,144.60,1.0,1.0,20.23,2.0,1.0,1.51,39.07,...,1.0,0.0,0.0,2.0,2.44,2.56,135.53,0.144219,-0.373980,50.0
3,C#CC(OC(N)=O)C1=CC=CC=C1,1,175.19,2.0,1.0,52.32,2.0,1.0,1.46,48.68,...,3.0,1.0,0.0,3.0,2.08,2.54,166.11,0.405593,-0.428173,66.0
4,C#CC1(OCC(O)CN2CCN(C3=CC=C(F)C=C3)CC2)CCCCC1,1,360.47,4.0,1.0,35.94,6.0,1.0,2.66,101.66,...,4.0,1.0,0.0,5.0,1.77,2.38,331.24,0.128054,-0.389382,142.0


### GraphDescriptors module

In [ ]:
for i, row in df.iterrows():
    mol = Chem.MolFromSmiles(row.SMILES)
    df.at[i, 'BertzCT'] = round(GraphDescriptors.BertzCT(mol), 2)
    df.at[i, 'Kappa1'] = round(GraphDescriptors.Kappa1(mol), 2)
df.head()

,SMILES,Activity,MW,#HBA,#HBD,PSA,#ROTB,#ALERTS,MlogP,#MR,...,#AromaticHeterocycles,#Heteroatoms,Morgan2,Morgan3,HeavyAtomMW,MaxPartialCharge,MinPartialCharge,#ValenceElectrons,BertzCT,Kappa1
0,BrC(Br)Br,1,252.73,0.0,0.0,0.00,0.0,1.0,2.45,30.57,...,0.0,3.0,1.00,1.00,251.72,0.124221,-0.063717,26.0,8.00,5.44
1,C#CC(C)(O)CC,1,98.14,1.0,1.0,20.23,1.0,1.0,0.78,29.75,...,0.0,1.0,2.29,2.29,88.06,0.121725,-0.377933,40.0,86.84,6.52
2,C#CC(O)(/C=C/Cl)CC,1,144.60,1.0,1.0,20.23,2.0,1.0,1.51,39.07,...,0.0,2.0,2.44,2.56,135.53,0.144219,-0.373980,50.0,145.49,8.55
3,C#CC(OC(N)=O)C1=CC=CC=C1,1,175.19,2.0,1.0,52.32,2.0,1.0,1.46,48.68,...,0.0,3.0,2.08,2.54,166.11,0.405593,-0.428173,66.0,326.53,9.14
4,C#CC1(OCC(O)CN2CCN(C3=CC=C(F)C=C3)CC2)CCCCC1,1,360.47,4.0,1.0,35.94,6.0,1.0,2.66,101.66,...,0.0,5.0,1.77,2.38,331.24,0.128054,-0.389382,142.0,599.43,19.16


### Avalon

In [ ]:
def generate_AVfpts(data):
    Avalon_fpts = []
    mols = [Chem.MolFromSmiles(x) for x in data if x is not None]
    for mol in tqdm(mols):
        avfpts = pyAvalonTools.GetAvalonFP(mol, nBits=512)
        Avalon_fpts.append(avfpts)
    return np.array(Avalon_fpts)

In [ ]:
Avalon_fpts = generate_AVfpts(df['SMILES'])

100%|██████████| 6748/6748 [00:08<00:00, 817.58it/s]


In [ ]:
Avalon_fpts.shape

(6748, 512)

In [ ]:
for i, row in df.iterrows():
    for j in range(Avalon_fpts.shape[1]):
        df.at[i, f'A_{j+1}'] = Avalon_fpts[i][j]

df.head()

<ipython-input-14-7ed5c3b00665>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[i, f'A_{j+1}'] = Avalon_fpts[i][j]
<ipython-input-14-7ed5c3b00665>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[i, f'A_{j+1}'] = Avalon_fpts[i][j]
<ipython-input-14-7ed5c3b00665>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `new

,SMILES,Activity,MW,#HBA,#HBD,PSA,#ROTB,#ALERTS,MlogP,#MR,...,A_503,A_504,A_505,A_506,A_507,A_508,A_509,A_510,A_511,A_512
0,BrC(Br)Br,1,252.73,0.0,0.0,0.00,0.0,1.0,2.45,30.57,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,C#CC(C)(O)CC,1,98.14,1.0,1.0,20.23,1.0,1.0,0.78,29.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,C#CC(O)(/C=C/Cl)CC,1,144.60,1.0,1.0,20.23,2.0,1.0,1.51,39.07,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,C#CC(OC(N)=O)C1=CC=CC=C1,1,175.19,2.0,1.0,52.32,2.0,1.0,1.46,48.68,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,C#CC1(OCC(O)CN2CCN(C3=CC=C(F)C=C3)CC2)CCCCC1,1,360.47,4.0,1.0,35.94,6.0,1.0,2.66,101.66,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [ ]:
del df['SMILES']

### Feature selection

In [ ]:
def select_features(X_train, y_train):
    fs = SelectKBest(score_func=mutual_info_classif, k=401) # k equals to approximately 75% of total features
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    return X_train_fs, fs

In [ ]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [ ]:
X_fs, fs = select_features(X, y)

In [ ]:
cols_idxs = fs.get_support(indices=True)
df = df.iloc[:, cols_idxs]

In [ ]:
df.head()

,Activity,MW,#HBA,#HBD,PSA,#ROTB,#ALERTS,MlogP,#MR,#HeavyAtoms,...,A_501,A_502,A_503,A_504,A_505,A_506,A_507,A_509,A_510,A_511
0,1,252.73,0.0,0.0,0.00,0.0,1.0,2.45,30.57,4.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,98.14,1.0,1.0,20.23,1.0,1.0,0.78,29.75,7.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,144.60,1.0,1.0,20.23,2.0,1.0,1.51,39.07,9.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1,175.19,2.0,1.0,52.32,2.0,1.0,1.46,48.68,13.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1,360.47,4.0,1.0,35.94,6.0,1.0,2.66,101.66,26.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
df.to_csv('avalon_classification_actual.csv', index=False)